In [2]:
import tensorflow as tf
from simple_cnn_to_mnist import CNN_MNIST_MODEL
import numpy as np
import math

In [3]:
def read_tf_iterator(path):
    images = []
    labels = []
    eyes_mat = np.eye(10)
    for record in tf.python_io.tf_record_iterator(path):
        ex = tf.train.Example()
        ex.ParseFromString(record)
        raw = ex.features.feature["raw"].bytes_list.value[0]
        label = ex.features.feature["label"].int64_list.value[0]
        raw = np.frombuffer(raw,np.float32)
        images.append(raw)
        labels.append(eyes_mat[label])
    images = np.array(images)
    labels = np.array(labels)
    return images,labels

In [4]:
train_images,train_labels = read_tf_iterator("tfrecords/train.tf")
val_images,val_labels = read_tf_iterator("tfrecords/val.tf")
test_images,test_labels = read_tf_iterator("tfrecords/test.tf")

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [13]:
tf.reset_default_graph()
input_x = tf.placeholder(tf.float32,(None,784))
reshape_x = tf.reshape(input_x,(-1,28,28,1))
input_y = tf.placeholder(tf.float32,(None,10))
cnn_model = CNN_MNIST_MODEL(reshape_x,input_y,0.001,0.98,50)
prediction = cnn_model.forward()
train_op,loss,learning_rate,global_step,accuracy = cnn_model.train()

In [14]:
def get_batch(images,labels,batch_size):
    batch_index = 0
    total_count = min(len(images),len(labels))
    epoch = math.ceil(total_count / batch_size)
    while(True):
        start_index = batch_index*batch_size
        end_index = min((batch_index+1)*batch_size,total_count)
        batch_images = images[start_index:end_index]
        batch_labels = labels[start_index:end_index]
        yield batch_images,batch_labels
        batch_index = (batch_index + 1) % epoch

In [15]:
EPOCH = 3000
train_batch_size = 1024

get_batch_itor = get_batch(train_images,train_labels,train_batch_size)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in np.arange(EPOCH):
        train_batch_images,train_batch_labels = get_batch_itor.__next__()
        sess.run(train_op,feed_dict={input_x:train_batch_images,input_y:train_batch_labels})
        if (epoch+1) % 100 == 0:
            ls,lr,gs,acc = sess.run([loss,learning_rate,global_step,accuracy],feed_dict={input_x:val_images,input_y:val_labels})
            print("Val[{}/{}]: loss={}, learning_rate={}, accuracy = {}, global_step={}".format(epoch+1,EPOCH,ls,lr,acc,gs))
    ls,lr,gs,acc = sess.run([loss,learning_rate,global_step,accuracy],feed_dict={input_x:test_images,input_y:test_labels})
    print("Test[{}/{}]: loss={}, learning_rate={}, accuracy = {}, global_step={}".format(epoch+1,EPOCH,ls,lr,acc,gs))

Val[100/3000]: loss=545.0377807617188, learning_rate=0.0009604000370018184, accuracy = 0.9685999751091003, global_step=100
Val[200/3000]: loss=340.0621032714844, learning_rate=0.0009223681990988553, accuracy = 0.9800000190734863, global_step=200
Val[300/3000]: loss=269.81488037109375, learning_rate=0.0008858424844220281, accuracy = 0.9846000075340271, global_step=300
Val[400/3000]: loss=236.34603881835938, learning_rate=0.0008507631719112396, accuracy = 0.9864000082015991, global_step=400
Val[500/3000]: loss=209.76968383789062, learning_rate=0.0008170729852281511, accuracy = 0.9878000020980835, global_step=500
Val[600/3000]: loss=238.28707885742188, learning_rate=0.0007847169181331992, accuracy = 0.9861999750137329, global_step=600
Val[700/3000]: loss=223.6804962158203, learning_rate=0.0007536421762779355, accuracy = 0.9869999885559082, global_step=700
Val[800/3000]: loss=217.61764526367188, learning_rate=0.0007237979443743825, accuracy = 0.9882000088691711, global_step=800
Val[900/300